In [1]:
# Colab setup: install minimal deps
!pip -q install numpy pandas pynacl

# Folders
import os, json, time, math, base64, hashlib, random
from datetime import datetime, timedelta
os.makedirs("data", exist_ok=True)
os.makedirs("model", exist_ok=True)
os.makedirs("out", exist_ok=True)

print("Ready. Folders: /data, /model, /out")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.3 MB/s eta 0:00:00
Ready. Folders: /data, /model, /out


In [2]:
import json, hashlib, time
from typing import Dict, Any

def now_ts() -> int:
    return int(time.time())

def sha256_hex(b: bytes) -> str:
    return hashlib.sha256(b).hexdigest()

def canonical_json(d: Dict[str, Any]) -> bytes:
    # Stable, minimal JSON representation for signing
    return json.dumps(d, separators=(",", ":"), sort_keys=True).encode()

def model_hash(weights: Dict[str, Any]) -> str:
    blob = canonical_json({"w": [float(x) for x in weights["w"]], "b": float(weights["b"])})
    return "0x" + sha256_hex(blob)

def make_pseudo_cid(payload: Dict[str, Any]) -> str:
    # Local, IPFS-free stand-in (good enough for hackathon demos)
    return f"cid_sha256:{sha256_hex(canonical_json(payload))}"

In [3]:
import pandas as pd
import numpy as np

def load_cashflow_csv(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, parse_dates=["date"])
    df = df.sort_values("date").reset_index(drop=True)
    return df

def daily_series(df: pd.DataFrame, days: int = 90) -> pd.DataFrame:
    end = df["date"].max()
    start = end - pd.Timedelta(days=days-1)
    rng = pd.date_range(start, end, freq="D")
    s = df.set_index("date")["amount"].groupby(pd.Grouper(freq="D")).sum().reindex(rng, fill_value=0.0)
    return pd.DataFrame({"date": rng, "amount": s.values})

def max_drawdown(series: np.ndarray) -> float:
    cum = series.cumsum()
    peak = np.maximum.accumulate(cum)
    return float((peak - cum).max())

def periodicity_score(series: np.ndarray) -> float:
    # crude autocorr at lags 14, 28, 30 (pay cycles)
    def acf(x, lag):
        x1 = x[:-lag]; x2 = x[lag:]
        if x1.std() < 1e-8 or x2.std() < 1e-8: return 0.0
        return float(np.corrcoef(x1, x2)[0,1])
    if len(series) < 31: return 0.0
    return max(0.0, max(acf(series, l) for l in [14,28,30]))

def shock_recovery(series: np.ndarray) -> float:
    # Approx. steps to recover from 1-σ negative shock; normalized to [0,1]
    x = series.copy()
    if x.std() < 1e-8: return 1.0
    shock = -x.std()
    bal = x.cumsum(); target = bal[-1]
    bal2 = bal + shock
    rec = next((i+1 for i in range(len(bal2)) if bal2[i] >= target), len(bal2))
    return float(min(rec / len(bal2), 1.0))

def feature_vector(df: pd.DataFrame, window_days: int = 90) -> np.ndarray:
    ds = daily_series(df, days=window_days)
    x = ds["amount"].values.astype(float)
    inflow  = np.clip(x, 0, None)
    outflow = np.clip(-x, 0, None)
    feats = []
    # Level & volatility
    feats += [inflow.mean(), inflow.std() + 1e-6, outflow.mean(), outflow.std() + 1e-6]
    # Net and volatility ratio
    net = inflow.mean() - outflow.mean()
    feats += [net, (outflow.std() / (inflow.std() + 1e-6))]
    # Drawdown on cumulative
    feats += [max_drawdown(x)]
    # Periodicity
    feats += [periodicity_score(inflow - outflow)]
    # Shock recovery
    feats += [shock_recovery(x)]
    # Skewness / kurtosis proxies
    feats += [float(pd.Series(x).skew()), float(pd.Series(x).kurt())]
    feats = np.array(feats, dtype=float)
    # log-scale money-like magnitudes for stability
    money_idx = [0,1,2,3,4,5,6]
    feats[money_idx] = np.sign(feats[money_idx]) * np.log1p(np.abs(feats[money_idx]))
    return feats  # shape (p,)


In [4]:
import numpy as np
from typing import Dict, Tuple, List

def init_model(p: int, seed: int = 0) -> Dict[str, np.ndarray]:
    rng = np.random.default_rng(seed)
    w = rng.normal(0, 0.05, size=(p,)).astype(float)
    b = 0.0
    return {"w": w, "b": b}

def sigmoid(z): return 1.0 / (1.0 + np.exp(-z))

def predict_proba(weights, X):
    return sigmoid(X @ weights["w"] + weights["b"])

def loss_and_grad(weights, X, y, l2=1e-4):
    p = predict_proba(weights, X)
    eps = 1e-8
    loss = -(y*np.log(p+eps)+(1-y)*np.log(1-p+eps)).mean() + 0.5*l2*np.sum(weights["w"]**2)
    g = (p - y) / len(y)
    grad_w = X.T @ g + l2*weights["w"]
    grad_b = g.sum()
    return float(loss), {"w": grad_w, "b": grad_b}

def clip_grad(grad, max_norm=1.0):
    norm = float(np.sqrt((grad["w"]**2).sum() + grad["b"]**2))
    if norm > max_norm:
        scale = max_norm / (norm + 1e-8)
        grad["w"] *= scale; grad["b"] *= scale
    return grad

def add_noise(grad, sigma=0.0, rng=None):
    if sigma <= 0: return grad
    rng = rng or np.random.default_rng()
    grad["w"] = grad["w"] + rng.normal(0, sigma, size=grad["w"].shape)
    grad["b"] = grad["b"] + rng.normal(0, sigma)
    return grad

def fedavg_round(global_weights, client_data: List[Tuple[np.ndarray, np.ndarray]],
                 lr=0.05, l2=1e-4, clip=1.0, sigma=0.05, rng=None):
    grads, losses = [], []
    for (X, y) in client_data:
        loss, grad = loss_and_grad(global_weights, X, y, l2=l2)
        grad = clip_grad(grad, max_norm=clip)
        grad = add_noise(grad, sigma=sigma, rng=rng)
        grads.append(grad); losses.append(loss)
    g_w = sum(g["w"] for g in grads) / len(grads)
    g_b = sum(g["b"] for g in grads) / len(grads)
    return {"w": global_weights["w"] - lr*g_w, "b": global_weights["b"] - lr*g_b}, float(np.mean(losses))

In [5]:
import numpy as np, pandas as pd

def simulate_client(days=120, seed=0, risk=0.3):
    rng = np.random.default_rng(seed)
    start = pd.Timestamp.today().normalize() - pd.Timedelta(days=days)
    dates = [start + pd.Timedelta(days=i+1) for i in range(days)]
    # inflow: periodic pay + small noise
    pay_period = rng.choice([14, 28, 30], p=[0.2,0.5,0.3])
    inflow = np.zeros(days)
    for i in range(days):
        if (i % pay_period) == 0:
            inflow[i] += rng.normal(100, 20)
        inflow[i] += max(0, rng.normal(5, 5))
    # outflow: daily spend + shocks depending on risk
    outflow = np.maximum(0, rng.normal(5 + 20*risk, 5 + 10*risk, size=days))
    if rng.random() < 0.4:
        j = rng.integers(10, days-10)
        outflow[j:j+3] += rng.normal(60, 20, size=3)
    amounts = inflow - outflow
    df = pd.DataFrame({"date": dates, "amount": amounts})
    # label proxy: default in next 30d ~ f(vol, drawdown, risk)
    bal = df["amount"].cumsum()
    vol = float(np.std(df["amount"]))
    dd  = float(np.max(np.maximum.accumulate(bal) - bal))
    pd_prob = 1/(1+np.exp(-( -0.5 + 0.03*vol + 0.02*dd + 1.2*risk )))
    label = 1 if rng.random() < pd_prob else 0
    return df, int(label)

# Generate a small federation
def generate_dataset(n_clients=5, days=120, seed=42, outdir="data"):
    rng = np.random.default_rng(seed)
    metas = []
    for i in range(1, n_clients+1):
        risk = float(rng.uniform(0.05, 0.6))
        df, label = simulate_client(days=days, seed=seed+i, risk=risk)
        path = f"{outdir}/client_{i}.csv"
        df.to_csv(path, index=False)
        metas.append({"client_id": i, "path": path, "label": label, "risk": risk})
    pd.DataFrame(metas).to_csv(f"{outdir}/meta.csv", index=False)
    print(f"Created {n_clients} client CSVs in /data and meta.csv")

# Run generator
generate_dataset(n_clients=5, days=120, seed=42, outdir="data")


Created 5 client CSVs in /data and meta.csv


In [6]:
import os, json, numpy as np, pandas as pd

def load_clients(datadir="data"):
    meta = pd.read_csv(f"{datadir}/meta.csv")
    Xs, ys = [], []
    for _, row in meta.iterrows():
        df = load_cashflow_csv(row["path"])
        x = feature_vector(df)
        y = np.array([row["label"]], dtype=float)  # one label per client
        Xs.append(x.reshape(1,-1))
        ys.append(y)
    return Xs, ys

def evaluate(W, Xs, ys):
    X = np.vstack(Xs)
    y = np.concatenate(ys)
    p = predict_proba(W, X).reshape(-1)
    brier = float(np.mean((p - y)**2))
    acc = float(np.mean(((p>=0.5).astype(int) == y)))
    return {"Brier": brier, "Acc": acc}

# Train
Xs, ys = load_clients("data")
p = Xs[0].shape[1]
W = init_model(p, seed=0)
losses = []
rng = np.random.default_rng(0)
rounds, lr, l2, clip, sigma = 8, 0.1, 1e-4, 1.0, 0.05

for r in range(1, rounds+1):
    W, loss = fedavg_round(W, list(zip(Xs, ys)), lr=lr, l2=l2, clip=clip, sigma=sigma, rng=rng)
    losses.append(loss)
    print(f"Round {r}: loss={loss:.4f}")

metrics = evaluate(W, Xs, ys)
print("Metrics:", metrics)

# Save global model
out = {
  "weights": {"w": W["w"].tolist(), "b": float(W["b"])},
  "metrics": metrics,
  "rounds": rounds,
  "dp_sigma": sigma,
  "clip": clip,
  "lr": lr,
  "l2": l2
}
out["model_hash"] = sha256_hex(json.dumps(out["weights"], sort_keys=True).encode())
os.makedirs("model", exist_ok=True)
with open("model/global_model.json", "w") as f:
    json.dump(out, f, indent=2)
print("Saved model → model/global_model.json")
print("model_hash = 0x" + out["model_hash"])


Round 1: loss=0.7543
Round 2: loss=0.1989
Round 3: loss=0.0490
Round 4: loss=0.0190
Round 5: loss=0.0127
Round 6: loss=0.0099
Round 7: loss=0.0086
Round 8: loss=0.0075
Metrics: {'Brier': 9.280123651198106e-05, 'Acc': 1.0}
Saved model → model/global_model.json
model_hash = 0x9ca8681d80274bc25c537d0193a02d92f8ac08e2d78204ac033040edbb72b542


In [7]:
import json, time, base64, numpy as np, pandas as pd
from nacl.signing import SigningKey, VerifyKey

def load_model(path="model/global_model.json"):
    with open(path, "r") as f:
        obj = json.load(f)
    w = np.array(obj["weights"]["w"], dtype=float)
    b = float(obj["weights"]["b"])
    return {"w": w, "b": b}, obj

def score_csv(csv_path: str, model_path="model/global_model.json"):
    W, meta = load_model(model_path)
    df = load_cashflow_csv(csv_path)
    x = feature_vector(df).reshape(1,-1)
    pd_prob = float(sigmoid(x @ W["w"] + W["b"]))  # P(default in 30d)
    cri = float(1.0 - pd_prob)                     # score ∈ [0,1]
    return cri, pd_prob, meta

# Deterministic signer (seeded for reproducibility in demos)
seed_bytes = hashlib.sha256(b"demo_attestor_seed").digest()
SK = SigningKey(seed_bytes)
PK = SK.verify_key

# Choose a client file to score (client_1.csv) and a demo wallet address
addr = "ADDR_DEMO_ABC123"
csv_path = "data/client_1.csv"
epoch = 1
metric_id = "CRI"

score, pd_prob, meta = score_csv(csv_path)
att = {
  "attestation_id": f"att-{sha256_hex((addr + csv_path + str(now_ts())).encode())[:16]}",
  "subject_type": "user_wallet",
  "subject_id": addr,
  "metric_id": metric_id,
  "score": round(score, 6),
  "uncertainty": 0.08,           # placeholder
  "epoch": epoch,
  "model_hash": "0x" + meta["model_hash"],
  "dp_epsilon": 1.2,             # config placeholder
  "dp_delta": 1e-5,
  "consortium_id": "demo",
  "metrics": meta.get("metrics", {}),
  "issued_at": now_ts(),
  "expires_at": now_ts() + 30*24*3600,
  "artifact_cids": {"model_card": "", "explain": ""}
}

# Sign canonical JSON hash with Ed25519 (Algorand-compatible curve)
preimage = canonical_json(att)
h = hashlib.sha256(preimage).digest()
sig = SK.sign(h).signature
sig_b64 = base64.b64encode(sig).decode()
pk_b64  = base64.b64encode(bytes(PK)).decode()

cid = make_pseudo_cid(att)  # local pseudo-CID for demos

bundle = {
  "attestation": att,
  "cid": cid,
  "sig_b64": sig_b64,
  "attestor_pk_b64": pk_b64,
  "app_args": {
    "subject": addr,
    "metric": metric_id,
    "score_u16": int(round(score * 10000)),
    "cid": cid,
    "epoch": epoch,
    "sig_b64": sig_b64
  }
}

with open(f"out/attestation_{metric_id}_{addr[:6]}.json", "w") as f:
    json.dump(bundle, f, indent=2)

print(json.dumps(bundle["attestation"], indent=2))
print("\nApp args (copy to contract call):")
print(json.dumps(bundle["app_args"], indent=2))
print("\nSaved →", f"out/attestation_{metric_id}_{addr[:6]}.json")


{
  "attestation_id": "att-852e32ed26f0b1bf",
  "subject_type": "user_wallet",
  "subject_id": "ADDR_DEMO_ABC123",
  "metric_id": "CRI",
  "score": 0.00159,
  "uncertainty": 0.08,
  "epoch": 1,
  "model_hash": "0x9ca8681d80274bc25c537d0193a02d92f8ac08e2d78204ac033040edbb72b542",
  "dp_epsilon": 1.2,
  "dp_delta": 1e-05,
  "consortium_id": "demo",
  "metrics": {
    "Brier": 9.280123651198106e-05,
    "Acc": 1.0
  },
  "issued_at": 1760832795,
  "expires_at": 1763424795,
  "artifact_cids": {
    "model_card": "",
    "explain": ""
  }
}

App args (copy to contract call):
{
  "subject": "ADDR_DEMO_ABC123",
  "metric": "CRI",
  "score_u16": 16,
  "cid": "cid_sha256:539447937eaed7e1d80b5e7deaaeba1a56002356449e9c9b4239793756e89c8f",
  "epoch": 1,
  "sig_b64": "Q5UGyeWt6vvdVspul2d/fiX84ocqfzVWMJLyjDI/RE6DubPYn+ylW+P+Gc5myDR2VB8O8h0Whj3V66Ox8U3FAw=="
}

Saved → out/attestation_CRI_ADDR_D.json


/tmp/ipython-input-853037182.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pd_prob = float(sigmoid(x @ W["w"] + W["b"]))  # P(default in 30d)


In [8]:
from nacl.signing import VerifyKey

with open(f"out/attestation_{metric_id}_{addr[:6]}.json", "r") as f:
    b = json.load(f)

payload = b["attestation"]
sig_b64 = b["sig_b64"]
pk_b64  = b["attestor_pk_b64"]

canonical = canonical_json(payload)
h = hashlib.sha256(canonical).digest()
vk = VerifyKey(base64.b64decode(pk_b64))

try:
    vk.verify(h, base64.b64decode(sig_b64))
    print("✅ Signature valid over canonical JSON hash.")
except Exception as e:
    print("❌ Invalid signature:", e)

print("\nPseudo-CID (for storage demo):", b["cid"])
print("Score_u16 (fixed-point):", b["app_args"]["score_u16"])


✅ Signature valid over canonical JSON hash.

Pseudo-CID (for storage demo): cid_sha256:539447937eaed7e1d80b5e7deaaeba1a56002356449e9c9b4239793756e89c8f
Score_u16 (fixed-point): 16


In [9]:
# Self-contained demo: simulate data, train a federated logistic model, print a user's CRI score and contract app_args.
import numpy as np, pandas as pd, hashlib, json, time

# ---- Data simulation ----
def simulate_client(days=120, seed=0, risk=0.3):
    rng = np.random.default_rng(seed)
    start = pd.Timestamp.today().normalize() - pd.Timedelta(days=days)
    dates = [start + pd.Timedelta(days=i+1) for i in range(days)]
    pay_period = rng.choice([14, 28, 30], p=[0.2,0.5,0.3])
    inflow = np.zeros(days)
    for i in range(days):
        if (i % pay_period) == 0:
            inflow[i] += rng.normal(100, 20)
        inflow[i] += max(0, rng.normal(5, 5))
    outflow = np.maximum(0, rng.normal(5 + 20*risk, 5 + 10*risk, size=days))
    if rng.random() < 0.4:
        j = rng.integers(10, days-10)
        outflow[j:j+3] += rng.normal(60, 20, size=3)
    amounts = inflow - outflow
    df = pd.DataFrame({"date": dates, "amount": amounts})
    bal = df["amount"].cumsum()
    vol = float(np.std(df["amount"]))
    dd  = float(np.max(np.maximum.accumulate(bal) - bal))
    pd_prob = 1/(1+np.exp(-( -0.5 + 0.03*vol + 0.02*dd + 1.2*risk )))
    label = 1 if rng.random() < pd_prob else 0
    return df, int(label)

# ---- Feature engineering ----
def daily_series(df: pd.DataFrame, days: int = 90) -> pd.DataFrame:
    end = df["date"].max()
    start = end - pd.Timedelta(days=days-1)
    rng = pd.date_range(start, end, freq="D")
    s = df.set_index("date")["amount"].groupby(pd.Grouper(freq="D")).sum().reindex(rng, fill_value=0.0)
    return pd.DataFrame({"date": rng, "amount": s.values})

def max_drawdown(series: np.ndarray) -> float:
    cum = series.cumsum()
    peak = np.maximum.accumulate(cum)
    return float((peak - cum).max())

def periodicity_score(series: np.ndarray) -> float:
    def acf(x, lag):
        x1 = x[:-lag]; x2 = x[lag:]
        if x1.std() < 1e-8 or x2.std() < 1e-8: return 0.0
        return float(np.corrcoef(x1, x2)[0,1])
    if len(series) < 31: return 0.0
    return max(0.0, max(acf(series, l) for l in [14,28,30]))

def shock_recovery(series: np.ndarray) -> float:
    x = series.copy()
    if x.std() < 1e-8: return 1.0
    shock = -x.std()
    bal = x.cumsum(); target = bal[-1]
    bal2 = bal + shock
    rec = next((i+1 for i in range(len(bal2)) if bal2[i] >= target), len(bal2))
    return float(min(rec / len(bal2), 1.0))

def feature_vector(df: pd.DataFrame, window_days: int = 90) -> np.ndarray:
    ds = daily_series(df, days=window_days)
    x = ds["amount"].values.astype(float)
    inflow  = np.clip(x, 0, None)
    outflow = np.clip(-x, 0, None)
    feats = []
    feats += [inflow.mean(), inflow.std() + 1e-6, outflow.mean(), outflow.std() + 1e-6]
    net = inflow.mean() - outflow.mean()
    feats += [net, (outflow.std() / (inflow.std() + 1e-6))]
    feats += [max_drawdown(x)]
    feats += [periodicity_score(inflow - outflow)]
    feats += [shock_recovery(x)]
    feats += [float(pd.Series(x).skew()), float(pd.Series(x).kurt())]
    feats = np.array(feats, dtype=float)
    money_idx = [0,1,2,3,4,5,6]
    feats[money_idx] = np.sign(feats[money_idx]) * np.log1p(np.abs(feats[money_idx]))
    return feats

# ---- Federated logistic regression ----
def init_model(p: int, seed: int = 0):
    rng = np.random.default_rng(seed)
    return {"w": rng.normal(0, 0.05, size=p).astype(float), "b": 0.0}

def sigmoid(z): return 1.0 / (1.0 + np.exp(-z))
def predict_proba(W, X): return sigmoid(X @ W["w"] + W["b"])

def loss_and_grad(W, X, y, l2=1e-4):
    p = predict_proba(W, X); eps=1e-8
    loss = -(y*np.log(p+eps)+(1-y)*np.log(1-p+eps)).mean() + 0.5*l2*np.sum(W["w"]**2)
    g = (p - y) / len(y)
    return float(loss), {"w": X.T @ g + l2*W["w"], "b": g.sum()}

def clip_grad(g, max_norm=1.0):
    norm = float(np.sqrt((g["w"]**2).sum() + g["b"]**2))
    if norm > max_norm:
        scale = max_norm / (norm + 1e-8)
        g["w"] *= scale; g["b"] *= scale
    return g

def add_noise(g, sigma=0.05, rng=None):
    if sigma <= 0: return g
    rng = rng or np.random.default_rng()
    g["w"] = g["w"] + rng.normal(0, sigma, size=g["w"].shape)
    g["b"] = g["b"] + rng.normal(0, sigma)
    return g

def fedavg_round(W, client_data, lr=0.1, l2=1e-4, clip=1.0, sigma=0.05, rng=None):
    grads, losses = [], []
    for (X, y) in client_data:
        loss, g = loss_and_grad(W, X, y, l2=l2)
        g = clip_grad(g, max_norm=clip)
        g = add_noise(g, sigma=sigma, rng=rng)
        grads.append(g); losses.append(loss)
    g_w = sum(g["w"] for g in grads) / len(grads)
    g_b = sum(g["b"] for g in grads) / len(grads)
    return {"w": W["w"] - lr*g_w, "b": W["b"] - lr*g_b}, float(np.mean(losses))

# ---- Simulate federation and train ----
clients = [simulate_client(days=120, seed=100+i, risk=0.15+0.1*i) for i in range(5)]
Xs, ys = [], []
for df, label in clients:
    Xs.append(feature_vector(df).reshape(1,-1))
    ys.append(np.array([label], dtype=float))

W = init_model(Xs[0].shape[1], seed=0)
rng = np.random.default_rng(0)
for r in range(8):
    W, loss = fedavg_round(W, list(zip(Xs, ys)), lr=0.1, l2=1e-4, clip=1.0, sigma=0.05, rng=rng)

# ---- Score user 1 ----
pd_prob = float(predict_proba(W, Xs[0])[0])   # P(default)
cri = 1.0 - pd_prob

addr = "ADDR_DEMO_ABC123"
metric = "CRI"
app_args = {
    "subject": addr,
    "metric": metric,
    "score_u16": int(round(cri * 10000)),
    "cid": f"cid_demo_user1_{int(cri*10000)}",
    "epoch": 1,
    "sig_b64": ""
}

print({
    "user_address": addr,
    "prob_default_30d": round(pd_prob, 4),
    "CRI_score_0to1": round(cri, 4),
    "CRI_score_0to100": round(cri*100, 2),
    "app_args": app_args
})


{'user_address': 'ADDR_DEMO_ABC123', 'prob_default_30d': 0.9994, 'CRI_score_0to1': 0.0006, 'CRI_score_0to100': 0.06, 'app_args': {'subject': 'ADDR_DEMO_ABC123', 'metric': 'CRI', 'score_u16': 6, 'cid': 'cid_demo_user1_6', 'epoch': 1, 'sig_b64': ''}}


In [10]:
import os, json, hashlib
os.makedirs("model", exist_ok=True)

out = {
  "weights": {"w": W["w"].tolist(), "b": float(W["b"])},
  "metrics": metrics, # e.g., {"Brier": ..., "Acc": ...}
  "rounds": rounds,
  "dp_sigma": sigma, "clip": clip, "lr": lr, "l2": l2
}
out["model_hash"] = hashlib.sha256(json.dumps(out["weights"], sort_keys=True).encode()).hexdigest()
with open("model/global_model.json","w") as f:
    json.dump(out, f, indent=2)

print("Saved model → model/global_model.json")


Saved model → model/global_model.json
